**BASIC METHODOLOGY:**

Tool used: Google Collab, Google Drive to store the MRI Image file
Program used: Python, and libraries: mumpy, os, cv2, matplotlib, skimage

Input: The google collab mounts the google drive in its environment and reads the input file (Code shown below)

The algorithm basically covers some of the pre-processing (converting to grayscale) of the given MRI Image. Then it performs the threshold-segmentation.

In [ ]:
# Importing the namespace libraries

%matplotlib inline
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
from skimage.morphology import extrema
from skimage.segmentation import watershed as skwater

# I have put my image files in google drive. So mounting the google drive in my Google Collab environment
from google.colab import drive
drive.mount('/content/drive')

# Just testing to see the current directory location.
print(os.getcwd())

In [ ]:
# a function is defined to plot the images and compare the resulting segmentation

def PlotMyImage(image_title,img,ctype):
  plt.figure(figsize=(5, 5))
  # Plot the image based on its type
  if ctype=='bgr':
    # get blue, green, red
    b,g,r = cv2.split(img)
    # switch it to rgb
    rgb_img = cv2.merge([r,g,b])
    plt.imshow(rgb_img)
  elif ctype=='hsv':
    rgb = cv2.cvtColor(img,cv2.COLOR_HSV2RGB)
    plt.imshow(rgb)
  elif ctype=='gray':
    plt.imshow(img,cmap='gray')
  elif ctype=='rgb':
    plt.imshow(img)
  else:
    raise Exception("Unknown colour type")

  # Printing the x-axis and y-axis scales
  plt.axis('on')
  # Printing the title of the image
  plt.title(image_title)
  plt.show()

In [ ]:
#Read in the original image file from the google drive
orig_img = cv2.imread('./drive/MyDrive/Colab Notebooks/BMI734/hw1/mri_brain.jpg')

# Converting an image from one color space to another: RGB Color to Grayscale
gray_orig_img = cv2.cvtColor(orig_img,cv2.COLOR_BGR2GRAY)
PlotMyImage('Original Image - Brain with Skull',gray_orig_img,'gray')
print("\n\n");


In [ ]:
# We are plotting a histogram of the intensities in the grayscale image
# The data seems to be BiModal - distribution which means two clearly separate groups are visible in the histogram we plotted below
print("\n\n")
plt.hist(gray_orig_img.ravel(),256)
plt.show()

In [ ]:
# OTSU's Threshold (Ref. https://en.wikipedia.org/wiki/Otsu%27s_method) is now been implemented.
# It basically returns a single intensity threshold that separates pixels into two classes (foreground and background)
# The algorithm determines the tresholds by minimizing intra-class intensity variabce  (Such as by using the euclidian distance)
ret, thresh = cv2.threshold(gray_orig_img,0,255,cv2.THRESH_OTSU)
PlotMyImage('Applying Otsu',thresh,'gray')

In [ ]:
colormask = np.zeros(orig_img.shape, dtype=np.uint8)
colormask[thresh!=0] = np.array((0,0,255))
blended = cv2.addWeighted(orig_img,0.7,colormask,0.1,0)
PlotMyImage('Blended', blended, 'bgr')

In [ ]:
ret, markers = cv2.connectedComponents(thresh)

print(np.max(markers))

#Get the area taken by each component. Ignore label 0 since this is the background.
marker_area = [np.sum(markers==m) for m in range(np.max(markers)) if m!=0]

# The largest component by area
# We dropped zero above, so we add 1 here
largest_component = np.argmax(marker_area) + 1

# Now we extract pixels which correspond to the brain. (Note: Brain is the largest component)
brain_mask = markers==largest_component


# Make a copy of the original image
brain_out = orig_img.copy()

# In the copy of the original image we just created above,
# we are going to clear those pixels that don't correspond to the brain
brain_out[brain_mask==False] = (0,0,0)

# Now plot the image of brain only
PlotMyImage('Removed the Brain Cover skull - isolated the brain', brain_out,'rgb')